##Start | Declarations

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import csv

import IPython
from PIL import Image
import urllib
import random
import sqlite3

In [33]:
def get_data_from_index(index):
	return "Name: "+ df[df.index == index]["productDisplayName"].values[0] + "\nCategory: " + df[df.index == index]["masterCategory"].values[0] + "\nProduct Color: " + df[df.index == index]["baseColour"].values[0] + "\nProduct Type: " + df[df.index == index]['articleType'].values[0] + "\nUsage: " + df[df.index == index]["usage"].values[0]

def get_index_from_name(productDisplayName):
	return df[df.productDisplayName == productDisplayName]["index"].values[0]

def get_index_from_id(id):
	return df[df.id == id]["index"].values[0]

def get_id_from_index(index):
	return df[df.index == index]["id"].values[0]

def get_image_from_id(filename):
	return dfImages[dfImages.filename == filename]["link"].values[0]

def get_data_from_id(id):
	return "Name: "+ df[df.id == id]["productDisplayName"].values[0] + "\nCategory: " + df[df.id == id]["masterCategory"].values[0] + "\nProduct Color: " + df[df.id == id]["baseColour"].values[0] + "\nProduct Type: " + df[df.id == id]['articleType'].values[0] + "\nUsage: " + df[df.id == id]["usage"].values[0]


In [34]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Registration / Login Module


###Register

In [ ]:
# sqlite3 connection

con = sqlite3.connect('gdrive/My Drive/Colab Notebooks/Fashion Recommender/databases/fashion.db')

In [ ]:
# NOTE: Instead of making new Registration, use previous accounts, since their activities and recorded by default in the database

In [ ]:
# Use the username and password by running this cell and skip to "Login" section

query = "select username, password from users"
userValues = pd.read_sql_query(query, con)
userValues.head()

In [ ]:
print("--Register--")

username = input("Setup username (any string): ");
password = input("Setup password (any string): ");
id = random.randint(0, 1000)

--Register--
Setup username (any string): abc
Setup password (any string): abc123


In [ ]:
try:
  con.execute("select ProductID from users where userID = '{}'".format(username))
  print("User already exist!")
  username = None
  password = None
except:
  con.execute("insert into users values('{}', '{}', '{}')".format(id,username,password))
  print("Executed!")

con.commit()

In [ ]:
query = "select * from users"
personsValues = pd.read_sql_query(query, con)
personsValues.head()

,userID,username,password
0,1,abc,abc123


In [ ]:
con.commit()
con.close()

###Login

In [35]:
# sqlite3 connection

con = sqlite3.connect('gdrive/My Drive/Colab Notebooks/Fashion Recommender/databases/fashion.db')

In [36]:
# This is done so that a particular user's activities can be recalled for making perfect recommendations (Content-Filtering) 

print("--Login--")

username = input("Enter username: ");
password = input("Enter password: ");

--Login--
Enter username: abc
Enter password: abc123


In [37]:
try:
  ans = con.execute("select userID from users where username = '{}' and password = '{}'".format(username, password))
  print("Login Successful!")
except:
  print("Login Failed!")
  username = None
  password = None

tupleForm = ans.fetchone()
userID = tupleForm[0]

Login Successful!


In [38]:
con.commit()
con.close()

##Recommender

###Reading CSV 

In [39]:
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/Fashion Recommender/data/fashionDataset.csv', error_bad_lines=True)

In [40]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] # Removing unnamed/empty columns

In [41]:
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44441,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44442,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44443,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44444,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


### Features

In [45]:
# Features to be used for Recommendations

features = ['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']

In [46]:
# Combining selected features

for feature in features:
  df[feature] = df[feature].fillna("")
  df[feature] = df[feature].apply(str)

def combineFeatures(row):
  try:
    return row['gender'] + " " + row['masterCategory'] + " " + row['subCategory'] + " " + row['articleType'] + " " + row['baseColour'] + " " + row['season'] + " " + row['usage']
  except:
    print("Error:", row)  

df['combinedFeatures'] = df.apply(combineFeatures, axis=1)


In [47]:
## Adding index column to the dataset

i = -1

def indexNumber(row):
  global i
  i += 1
  return i  

df['index'] = df.apply(indexNumber, axis=1)

###Find user's previous transactions


In [48]:
# sqlite3 connection

con = sqlite3.connect('gdrive/My Drive/Colab Notebooks/Fashion Recommender/databases/fashion.db')

In [49]:
# Find if user has previously liked or purchased any fashion product

ans = con.execute("select productID from userPurchaseDetails where userID = '{}' and relation = 1".format(userID))
ans2 = con.execute("select productID from userPurchaseDetails where userID = '{}' and relation = 2".format(userID))
purchaseTuple = ans.fetchall()
likedTuple = ans2.fetchall()

In [ ]:
# Create List of all Products (Both Purchased and liked by user)

purchasedProductList = []
likedProductList = []

for singleValue in likedTuple:
  likedProductList.append(singleValue[0])

for singleValue in purchaseTuple:
  purchasedProductList.append(singleValue[0])

likedProductList

In [51]:
# features = ['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']


def get_Product_Features(productList, attribute):
  i = 0
  tempDict = {df.loc[df['id'] == productList[i], attribute].values[0]: 0}
  
  while (i < len(productList)):
    temp = df.loc[df['id'] == productList[i], attribute].values[0]
    if (temp in tempDict):
      tempDict[temp] = tempDict[temp] + 1
    else:  
      tempDict[temp] = 1

    i = i + 1
  return tempDict

# To get features of products liked by user (Saved in Cart or Wishlist)
liked_gender = get_Product_Features(likedProductList, 'gender')
liked_masterCategory = get_Product_Features(likedProductList, 'masterCategory')
liked_subCategory = get_Product_Features(likedProductList, 'subCategory')
liked_articleType = get_Product_Features(likedProductList, 'articleType')
liked_baseColour = get_Product_Features(likedProductList, 'baseColour')
liked_season = get_Product_Features(likedProductList, 'season')
liked_usage = get_Product_Features(likedProductList, 'usage')

# To get features of products purchased by user
purchased_gender = get_Product_Features(purchasedProductList, 'gender')
purchased_masterCategory = get_Product_Features(purchasedProductList, 'masterCategory')
purchased_subCategory = get_Product_Features(purchasedProductList, 'subCategory')
purchased_articleType = get_Product_Features(purchasedProductList, 'articleType')
purchased_baseColour = get_Product_Features(purchasedProductList, 'baseColour')
purchased_season = get_Product_Features(purchasedProductList, 'season')
purchased_usage = get_Product_Features(purchasedProductList, 'usage')

In [52]:
liked_masterCategory
liked_gender

{'Girls': 1, 'Men': 9, 'Unisex': 1, 'Women': 3}

In [53]:
# Calculating features of most liked product
liked_gender_max = max(liked_gender, key=liked_gender.get)
liked_masterCategory_max = max(liked_masterCategory, key=liked_masterCategory.get)
liked_subCategory_max = max(liked_subCategory, key=liked_subCategory.get)
liked_articleType_max = max(liked_articleType, key=liked_articleType.get)
liked_baseColour_max = max(liked_baseColour, key=liked_baseColour.get)
liked_season_max = max(liked_season, key=liked_season.get)
liked_usage_max = max(liked_usage, key=liked_usage.get)

# Calculating features of most purchased product
purchased_gender_max = max(purchased_gender, key=purchased_gender.get)
purchased_masterCategory_max = max(purchased_masterCategory, key=purchased_masterCategory.get)
purchased_subCategory_max = max(purchased_subCategory, key=purchased_subCategory.get)
purchased_articleType_max = max(purchased_articleType, key=purchased_articleType.get)
purchased_baseColour_max = max(purchased_baseColour, key=purchased_baseColour.get)
purchased_season_max = max(purchased_season, key=purchased_season.get)
purchased_usage_max = max(purchased_usage, key=purchased_usage.get)

In [54]:
# Combined Liked features
likedFeatures = liked_gender_max + " " + liked_masterCategory_max + " " + liked_subCategory_max + " " + liked_articleType_max + " " + liked_baseColour_max + " " + liked_season_max + " " + liked_usage_max
print(likedFeatures)

# Combined purchased features
purchasedFeatures = purchased_gender_max + " " + purchased_masterCategory_max + " " + purchased_subCategory_max + " " + purchased_articleType_max + " " + purchased_baseColour_max + " " + purchased_season_max + " " + purchased_usage_max
print(purchasedFeatures)

Men Footwear Topwear Sports Shoes Grey Summer Casual
Men Apparel Topwear Sweatshirts White Fall Casual


In [55]:
# Adding calculated liked and Purchased product features in Dataframe

df.loc[len(df.index)] = [0, liked_gender_max, liked_masterCategory_max, liked_subCategory_max, liked_articleType_max, liked_baseColour_max, liked_season_max, 2011.0, liked_usage_max, "NA", likedFeatures, len(df.index)]
df.loc[len(df.index)] = [1, purchased_gender_max, purchased_masterCategory_max, purchased_subCategory_max, purchased_articleType_max, purchased_baseColour_max, purchased_season_max, 2011.0, purchased_usage_max, "NA", purchasedFeatures, len(df.index)]

In [56]:
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,combinedFeatures,index
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,Men Apparel Topwear Shirts Navy Blue Fall Casual,0
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,Men Apparel Bottomwear Jeans Blue Summer Casual,1
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,Women Accessories Watches Watches Silver Winte...,2
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,Men Apparel Bottomwear Track Pants Black Fall ...,3
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,Men Apparel Topwear Tshirts Grey Summer Casual,4
...,...,...,...,...,...,...,...,...,...,...,...,...
44443,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt,Men Apparel Topwear Tshirts Blue Fall Casual,44443
44444,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume,Women Personal Care Fragrance Perfume and Body...,44444
44445,51623,Women,Accessories,Watches,Watches,Pink,Winter,2016.0,Casual,Fossil Women Pink Dial Chronograph Watch ES3050,Women Accessories Watches Watches Pink Winter ...,44445
44446,0,Men,Footwear,Topwear,Sports Shoes,Grey,Summer,2011.0,Casual,NA,Men Footwear Topwear Sports Shoes Grey Summer ...,44446


In [57]:
con.commit()
con.close()

### Get Cosine Similarity

In [58]:
# Vectorizing all combined features

hv = HashingVectorizer()
count_matrix = hv.fit_transform(df['combinedFeatures'])

likedProductID = len(df.index)-2
purchasedProductID = len(df.index)-1


In [59]:
# Making matrix start index of count matrix

def get_pos_nums(num):
  pos_nums = []
  while num != 0:
    pos_nums.append(num % 10)
    num = num // 10   
  return pos_nums

def round(num): 
  strNum = '0'
  i = 3
  j = 0
  mainNumStr = ""
  while (i < len(pos_nums)):
    mainNumStr = mainNumStr + str(pos_nums[len(pos_nums)-1-j])
    i=i+1
    j=j+1
  j = 0
  while (j < 3):
    mainNumStr = mainNumStr + strNum
    j = j + 1
  return int(mainNumStr)

if (likedProductID <= 1000):
  start = 0
else:
  # Function for getting number at 100th of 1000th place, e.g: productID = 12920 will give pos_nums = [0, 2, 9, 2, 1] as output 
  pos_nums = get_pos_nums(likedProductID)
  # Function for getting round figure e.g 3231 = 3000, 12332 = 12000, if value is less than 1000 then start from 0
  likedProductStart = round(pos_nums)

if (purchasedProductID <= 1000):
  start = 0
else:
  pos_nums = get_pos_nums(purchasedProductID)
  purchasedProductStart = round(pos_nums)

In [60]:
# Finding Similarity using Cosine similarity

size = 1000
df_len = df.shape[0]

def check_similarity(start, end):
  if (end > df_len):
    end = df_len 
  return cosine_similarity(count_matrix[start:end], count_matrix) 

# Getting similar products
likedProductSimilarity = check_similarity(likedProductStart, likedProductStart+size)
purchasedProductSimilarity = check_similarity(purchasedProductStart, purchasedProductStart+size)

similar_liked_products = list(enumerate(likedProductSimilarity[likedProductID-likedProductStart]))
similar_purchased_products = list(enumerate(purchasedProductSimilarity[purchasedProductID-purchasedProductStart]))

###Sorting products based on similarity and Printing Recommendations as Output

In [61]:
sorted_similar_liked_products = sorted(similar_liked_products, key=lambda x:x[1], reverse=True)
sorted_similar_purchased_products = sorted(similar_purchased_products, key=lambda x:x[1], reverse=True)

In [28]:
len(sorted_similar_liked_products)

44448

In [62]:
# Printing most similar

def printing_recommendations(productID, sorted_similar_products):
  i = 1
  n = 20
  print("\n-----TOP 20 RECOMMENDATIONS-----\n")
  for product in sorted_similar_products:
    if (product[0] != productID and product[0] != 0 or product[0] != 1):
      print("\nSr no: " + str(i))
      print(get_data_from_index(product[0]))
      i=i+1
      if i>n:
        more = input("\nShow more 20 products(type: y/n): ")
        if (more == 'y' or more == 'Y'):
          n = n + 20
          continue;
        else:
          print("\nProducts list terminated!!")
          break;
    else:
      continue;




In [ ]:
# Top 20 fashion products based on users previously liked products (Liked or Saved in wishlist or cart)

printing_recommendations(likedProductID, sorted_similar_liked_products)

In [ ]:
# Top 20 fashion products based on users previously purchased products (checked out and delivered)

printing_recommendations(purchasedProductID, sorted_similar_purchased_products)